# Napelemek temelésének előrejelzése gépi tanulási algoritmusok segítségével
## A feladat
`count félrevezet; cov,corr INF-et ad; quantile-t nem tudom használni,mint nyugodtan kivehetem mert úgyis nulla?

In [1]:
import math
import pandas as pd
import numpy as np

PATH_TO_TRAIN = '../data/raw/train15.csv'
DATE_FORMAT = '%Y%m%d %H:%M'
INDEX_COLUMN = 'TIMESTAMP'
TARGET_COLUMN = 'POWER'
feature_columns = ['VAR78', 'VAR79', 'VAR134', 'VAR157', 'VAR164',
       'VAR165', 'VAR166', 'VAR167','VAR169', 'VAR175', 'VAR178', 'VAR228']
ACCUMLATED_FEATURE_COLUMNS = ['VAR169', 'VAR175', 'VAR178', 'VAR228']

ONE_DAY = 24
ONE_WEEK = 7 * ONE_DAY
ONE_MONTH = 30* ONE_DAY
ONE_YEAR = 365 * ONE_DAY
PREDICT_INTERVAL = ONE_DAY * 5 

df_original = pd.read_csv(PATH_TO_TRAIN)

df = df_original.copy()
df = df[df['ZONEID'] == 1]
dateparse = lambda x: pd.datetime.strptime(x, DATE_FORMAT)
df[INDEX_COLUMN] = df[INDEX_COLUMN].apply(dateparse)

In [2]:
# Magyarázó változók hozzáadása a modellhez
PATH_TO_PREDICTORS = '../data/raw/predictors15.csv'

df_features = pd.read_csv(PATH_TO_PREDICTORS)
df_features[INDEX_COLUMN] = df_features[INDEX_COLUMN].apply(dateparse)

df = df.merge(df_features, how='left', on=[INDEX_COLUMN,'ZONEID'])

In [3]:
df["MONTH"] = df[INDEX_COLUMN].apply(lambda x: x.month)
df["HOUR"] = df[INDEX_COLUMN].apply(lambda x: x.hour)
# for i in np.arange(1,13):
#     df["MONTH"+str(i)]= df["TIMESTAMP"].apply(lambda x: x.month == i)
# for i in np.arange(24):
#     df["HOUR"+str(i)] = df["TIMESTAMP"].apply(lambda x: x.hour == i)

In [4]:
# Zóna konvertálása kategorikus változóvá
df["ZONE_1"] = df["ZONEID"].apply(lambda x: x == 1)
df["ZONE_2"] = df["ZONEID"].apply(lambda x: x == 2)
df["ZONE_3"] = df["ZONEID"].apply(lambda x: x == 3)
df = df.drop("ZONEID",axis=1)

# Timestamp legyen az index
df = df.set_index('TIMESTAMP')
#df = df.drop("TIMESTAMP",axis=1)

In [5]:
#Using one year to train and one month to predict
# df is the full data so we can calculate the rolling windows
TRAIN_SIZE = ONE_YEAR
df = df[:TRAIN_SIZE+PREDICT_INTERVAL]
y_test = df[TRAIN_SIZE:TRAIN_SIZE+PREDICT_INTERVAL][TARGET_COLUMN].copy() # ez mekkora szopás referencia

In [6]:
def clear_data_from_end(df, column, until):
    for i in range(until):
        df.iloc[-1*(i+1),df.columns.get_loc(column)] = None
    return df

def add_rolling(df,column, intervals, shift):
    for i in range(min(intervals),max(intervals)):
    #for i in intervals:
        if i >= shift:
            rolling_column = df[column].rolling(window = i)
            df["ROLLING_MEAN_"+column+"_"+str(i)] = rolling_column.mean().shift(shift)
            df["ROLLING_MIN_"+column+"_"+str(i)] = rolling_column.min().shift(shift)
            df["ROLLING_MAX_"+column+"_"+str(i)] = rolling_column.max().shift(shift)
            df["ROLLING_SUM_"+column+"_"+str(i)] = rolling_column.sum().shift(shift)
            df["ROLLING_MEDIAN_"+column+"_"+str(i)] = rolling_column.median().shift(shift)
            df["ROLLING_STD_"+column+"_"+str(i)] = rolling_column.std().shift(shift)
            df["ROLLING_VAR_"+column+"_"+str(i)] = rolling_column.var().shift(shift)
            df["ROLLING_SKEW_"+column+"_"+str(i)] = rolling_column.skew().shift(shift)
            df["ROLLING_KURT_"+column+"_"+str(i)] = rolling_column.kurt().shift(shift)
            #df[column] =  df[column].shift(shift)
    return df

def dissipate_features(df, column):
    return df[column].rolling(window=2).apply(lambda x: x[1] if x[1] - x[0] < 0 else x[1] - x[0])

In [7]:
from bokeh.plotting import figure,show
from bokeh.io import output_notebook

In [8]:
output_notebook()

thing_to_plot = df.copy()[:48][ACCUMLATED_FEATURE_COLUMNS]

for column in thing_to_plot:
    p = figure()
    p.line(np.arange(len(thing_to_plot)), thing_to_plot[column], legend="real")
    p.line(np.arange(len(thing_to_plot)), dissipate_features(thing_to_plot,column), legend="diss",color="orange")
    show(p)

Loading BokehJS ...

D:\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning


In [9]:
df = clear_data_from_end(df,TARGET_COLUMN,PREDICT_INTERVAL)
df = add_rolling(df,TARGET_COLUMN, [ONE_DAY, ONE_WEEK, ONE_MONTH], PREDICT_INTERVAL)

In [10]:
for column in ACCUMLATED_FEATURE_COLUMNS:
    df[column] = dissipate_features(df,column)

D:\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning


In [11]:
for column in feature_columns:
    df = clear_data_from_end(df,column,PREDICT_INTERVAL)
    df = add_rolling(df,column, [ONE_DAY, ONE_WEEK], PREDICT_INTERVAL)
    df = df.drop(column, axis = 1)

In [12]:
from xgboost import XGBRegressor #Ezt külön fel kellett rakni
from sklearn.svm import LinearSVR,SVR
from sklearn.svm import SVR

X_train = df[:TRAIN_SIZE] \
    .drop(TARGET_COLUMN,axis=1) \
    .dropna(axis=0)
y_train = df[:TRAIN_SIZE].dropna(axis=0)[TARGET_COLUMN]
    

xgb_model = XGBRegressor(nthread=4)
xgb_model.fit(X_train,y_train)

# For SVR we need Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
# Scale x and y (two scale objects)
x = sc_x.fit_transform(X_train)

svr_model = SVR()
svr_model.fit(x,y_train)

lsvr_model = LinearSVR()
lsvr_model.fit(x,y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [13]:
from sklearn.metrics import explained_variance_score

X_test = df[TRAIN_SIZE:TRAIN_SIZE+PREDICT_INTERVAL].drop(TARGET_COLUMN,axis=1)

print("XGBoost")
y_xgb = xgb_model.predict(X_test)
y_xgb[y_xgb < 0] = 0
print(explained_variance_score(y_xgb, y_test))

# For SVR we need Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
# Scale x and y (two scale objects)
x = sc_x.fit_transform(X_test)

print("LinearSVR")
y_lsvr =lsvr_model.predict(x)
y_lsvr[y_lsvr < 0] = 0
print(explained_variance_score(y_lsvr,y_test))

print("SVR")
y_svr = svr_model.predict(x)
y_svr[y_svr < 0] = 0
print(explained_variance_score(y_svr,y_test))

XGBoost
0.745093322901641
LinearSVR
0.26509630073683055
SVR
0.36716520562238353


In [14]:
from bokeh.plotting import figure,show
from bokeh.io import output_notebook

y_true = y_test.values

output_notebook()

p = figure()
p.circle(np.arange(PREDICT_INTERVAL), y_true, legend="real")
p.circle(np.arange(PREDICT_INTERVAL),y_xgb ,legend="XGBoost", color="orange")
p.circle(np.arange(PREDICT_INTERVAL),y_svr ,legend="SVR", color="green")
p.circle(np.arange(PREDICT_INTERVAL),y_lsvr ,legend="LinearSVR", color="purple")
show(p)

p = figure()
p.line(np.arange(PREDICT_INTERVAL), y_true, legend="real")
p.line(np.arange(PREDICT_INTERVAL),y_xgb ,legend="XGBoost", color="orange")
p.line(np.arange(PREDICT_INTERVAL),y_svr ,legend="SVR", color="green")
p.line(np.arange(PREDICT_INTERVAL),y_lsvr ,legend="LinearSVR", color="purple")
show(p)

Loading BokehJS ...

In [15]:
# from bokeh.plotting import figure, output_notebook, show
# output_notebook()
# for column in df:
#     print(column)
#     p = figure(plot_width=400, plot_height=400)
#     p.line(df.index[:24*5],df[column][:24*5])
#     show(p)

In [16]:
# Feature nélküli modellen javított amint beraktam a hónap, óra változókat numerikusan
# vajon mi lenne kategorikusan

In [17]:
#Jelentősen javított az akumlált értékek felhasználása

In [18]:
X_train.head()

,MONTH,HOUR,ZONE_1,ZONE_2,ZONE_3,ROLLING_MEAN_POWER_120,ROLLING_MIN_POWER_120,ROLLING_MAX_POWER_120,ROLLING_SUM_POWER_120,ROLLING_MEDIAN_POWER_120,...,ROLLING_KURT_VAR228_166,ROLLING_MEAN_VAR228_167,ROLLING_MIN_VAR228_167,ROLLING_MAX_VAR228_167,ROLLING_SUM_VAR228_167,ROLLING_MEDIAN_VAR228_167,ROLLING_STD_VAR228_167,ROLLING_VAR_VAR228_167,ROLLING_SKEW_VAR228_167,ROLLING_KURT_VAR228_167
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2012-05-05 23:00:00,5,23,True,False,False,0.158851,0.0,0.754744,19.062179,0.000288,...,70.918975,0.000019,0.0,0.000857,0.003135,0.0,0.000088,7.817795e-09,6.968388,55.429860
2012-05-06 00:00:00,5,0,True,False,False,0.158595,0.0,0.754744,19.031410,0.000288,...,77.736401,0.000016,0.0,0.000857,0.002707,0.0,0.000082,6.805258e-09,7.930904,71.354988
2012-05-06 01:00:00,5,1,True,False,False,0.158971,0.0,0.754744,19.076474,0.000288,...,79.909906,0.000015,0.0,0.000857,0.002475,0.0,0.000081,6.522562e-09,8.386963,78.213411
2012-05-06 02:00:00,5,2,True,False,False,0.159015,0.0,0.754744,19.081859,0.000288,...,80.077843,0.000014,0.0,0.000857,0.002348,0.0,0.000080,6.448357e-09,8.544383,80.400700
2012-05-06 03:00:00,5,3,True,False,False,0.159181,0.0,0.754744,19.101667,0.000288,...,80.113194,0.000014,0.0,0.000857,0.002309,0.0,0.000080,6.445752e-09,8.558176,80.570018
